In [15]:
# Import Dependencies
import pandas as pd
from census import Census
from config import census_key
#import cpi # this will be used to determine the cost of everything in today's dollars. cpi.inflate(amount,year of amount)

low_memory=False

In [12]:
# Import CSVs 
filepath = "Resources/"
kent_zipcodes = pd.read_csv(filepath+"kent_zipcodes.csv",squeeze=True,header=None)
oakland_zipcodes = pd.read_csv(filepath+"oakland_zipcodes.csv",squeeze=True,header=None)
saginaw_zipcodes = pd.read_csv(filepath+"saginaw_zipcodes.csv",squeeze=True,header=None)



In [17]:
# Get data from API for Census
c=Census(census_key)
census_data = c.acs5.get(("NAME", "B19013_001E", "B23025_004E", "B23025_005E", "B17001_002E", 
            "B17001A_002E","B17001B_002E", "B01003_001E",
            "B02001_002E", "B02001_003E","B25058_001E", "B25077_001E"
           ), {"for": "zip code tabulation area:*"})
# Convert to DataFrame
census_df = pd.DataFrame(census_data)
            
# Rename Columns
census_df = census_df.rename( 
        columns = { 
            "B19013_001E" : "Median Income", 
            "B23025_004E" : "Employed Count", 
            "B23025_005E" : "Unemployed Count", 
            "B17001_002E" : "Poverty Count", 
            "B17001A_002E" : "Poverty Count - White" , 
            "B17001B_002E" : "Poverty Count - Black" , 
            "B01003_001E" : "Total Population", 
            "B02001_002E" : "White Population", 
            "B02001_003E" : "Black Population", 
            "B25058_001E" : "Median Rent", 
            "B25077_001E" : "Median Home Value", 
            "NAME" : "Name",
            "zip code tabulation area": "Zipcode"
        }
)

census_df = census_df[
    [
        "Zipcode",
        "Total Population",
        "White Population",
        "Black Population",
        "Employed Count",
        "Unemployed Count",
        "Poverty Count",
        "Poverty Count - White",
        "Poverty Count - Black",
        "Median Income",
        "Median Rent",
        "Median Home Value",
    ]
]
census_df.head()

,Zipcode,Total Population,White Population,Black Population,Employed Count,Unemployed Count,Poverty Count,Poverty Count - White,Poverty Count - Black,Median Income,Median Rent,Median Home Value
0,29590,3543.0,718.0,2617.0,1104.0,46.0,869.0,177.0,663.0,30985.0,503.0,68600.0
1,93306,74296.0,51869.0,1727.0,28717.0,2379.0,15853.0,10291.0,796.0,54450.0,867.0,211100.0
2,93660,4082.0,2231.0,40.0,1313.0,229.0,1057.0,492.0,40.0,39625.0,597.0,171700.0
3,93110,15777.0,12042.0,311.0,7692.0,286.0,1332.0,943.0,100.0,93264.0,1877.0,844300.0
4,93212,22596.0,13033.0,2708.0,4484.0,575.0,3911.0,3263.0,111.0,42983.0,606.0,162100.0
